## Tasks


1. Download [docker image](https://hub.docker.com/r/djstrong/krnnt2) of KRNNT2. It includes the following tools:


    i. Morfeusz2 - morphological dictionary  
    ii. Corpus2 - corpus access library
    iii. Toki - tokenizer for Polish
    iv. Maca - morphosyntactic analyzer   
    v. KRNNT - Polish tagger

2. As an alternative you can use Tagger interfaces in [Clarin-Pl](https://ws.clarin-pl.eu/tager.shtml)
3. Use the tool to tag and lemmatize the law corpus.
4. Using the tagged corpus compute bigram statistic for the tokens containing:


    i. lemmatized, downcased word
    ii. morphosyntactic **category** of the word (`subst`, `fin`, `adj`, etc.)
   

In [2]:
import locale
import os
import string
import tarfile
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import regex
import operator
import pandas as pd


matplotlib.style.use("ggplot")
%matplotlib inline
locale.setlocale(locale.LC_COLLATE, "pl_PL.UTF-8")


'pl_PL.UTF-8'

In [1]:
# lematyzacja to sprowadzanie danego słowa do jego formy podstawowej (hasłowej), która reprezentuje dany wyraz, np. wiórkami → wiórek, jeżdżący → jeździć,
# tagging - the process of marking up a word in a text (corpus) as corresponding to a particular part of speech

import requests


r = requests.post('http://localhost:9200',data = "ma. kota, i skacze:")

r.text.split("\n")[0:1]
print(r.text)

['ma\tnone']

In [3]:



class Token():
    def __init__(self, lemma,morf,word) -> None:
        self.word: string = word # kota
        self.morf: string = morf
        self.lemma: string = lemma # kot
        self.flexeme : string = morf.split(":")[0] #morphosyntactic **category** of the word (`subst`, `fin`, `adj`, etc
    


def to_tokens(response):
    lines = response.text.split('\n')

    tokens = []
    token=None
    morf=None
    for line in lines:
        if not line.startswith('\t'):        
            word = line.split('\t')[0]
        else:
            lemma = line.split('\t')[1]
            if len(lemma.split(" ")) > 1:
                continue
            morf = line.split('\t')[2]
            tokens.append(Token(lemma,morf,word))
    return tokens

    

In [4]:

from tqdm.notebook import tqdm

i = 0
path = "../data/ustawy"

tokens = []

for filename in tqdm(os.listdir(path)):
    with open(os.path.join(path, filename), "r", encoding="utf-8") as file:
        act = file.read()
        act = regex.sub(r"\s+", " ", act)
        act = regex.sub(r"­", "", act)
        act = act.lower()

        response = requests.post('http://localhost:9200',data = act.encode('utf-8'))
        tokens+=to_tokens(response)
        
        i += 1
        
#         if i==50:
#             break


In [5]:
def bigrams(words):
    words = zip(words, words[1:])
    return [" ".join(pair) for pair in words]


In [6]:
text = [i.lemma   for i in tokens]
gram2 = bigrams(text)

# gram2 = [
#     token
#     for token in gram2
#     if all(char not in string.punctuation and not char.isdigit() for char in token)
# ]

Counter(gram2).most_common(5)


[('artykuł .', 83635),
 ('ustęp .', 53430),
 ('pozycja .', 45054),
 (', pozycja', 42999),
 ('. 1', 39961)]

In [7]:
text = [i.flexeme for i in tokens]
gram2 = bigrams(text)

# gram2 = [
#     i
#     for i in gram2
#     if not "interp" in i
# ]

Counter(gram2).most_common(5)

[('adj interp', 335747),
 ('interp adj', 326954),
 ('prep subst', 323894),
 ('subst adj', 304685),
 ('subst subst', 283592)]

5. Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries after computing the bigram counts.
6. For example: "Ala ma kota", which is tagged as:

   ```
   Ala	none
           Ala	subst:sg:nom:f	disamb
   ma	space
           mieć	fin:sg:ter:imperf	disamb
   kota	space
           kot	subst:sg:acc:m2	disamb
   .	none
           .	interp	disamb
   ```
   
   the algorithm should return the following bigrams: `ala:subst mieć:fin` and `mieć:fin kot:subst`.

In [8]:
def bigrams(tokens,filtered=True):
    token_pairs = zip(tokens, tokens[1:])
    return [(i,j) for i,j in token_pairs]


def isLetter(token):
    return all(char not in string.punctuation and not char.isdigit() for char in token.word)

    
gram2 = bigrams(tokens)
gram2 = [(i,j) for (i, j) in gram2 if isLetter(i) and isLetter(j) ]
gram2_tokens = gram2
gram2 = [f'{i.lemma}:{i.flexeme} {j.lemma}:{j.flexeme}' for (i, j) in gram2]

In [9]:
gram2_count =Counter(gram2)

In [10]:
gram2_count.most_common(10)

[('w:prep artykuł:brev', 31973),
 ('o:prep który:adj', 28658),
 ('który:adj mowa:subst', 28540),
 ('mowa:subst w:prep', 28473),
 ('w:prep ustęp:brev', 23501),
 ('z:prep dzień:subst', 11360),
 ('otrzymywać:fin brzmienie:subst', 10532),
 ('określić:ppas w:prep', 9781),
 ('do:prep sprawa:subst', 8718),
 ('ustawa:subst z:prep', 8625)]

7. Compute LLR statistic for this dataset.

In [11]:
from collections import defaultdict

token_count = defaultdict(int)

for bigram, count in gram2_count.items():
    (first_token, second_token) = bigram.split(" ")
    token_count[first_token] += count
    token_count[second_token] += count

total = sum(gram2_count.values())


In [12]:
def H(k):
    N = np.sum(k)
    return np.sum(k / N * np.ma.log(k / N).filled(0))


def llr(a, b):

    k11 = gram2_count[a + " " + b]
    k12 = token_count[b] - k11
    k21 = token_count[a] - k11
    k22 = total - k21 - k12 - k11
    k = np.array([[k11, k12], [k21, k22]])
    rowSums = np.sum(k, axis=1).tolist()
    colSums = np.sum(k, axis=0).tolist()

    return 2 * np.sum(k) * (H(k) - H(rowSums) - H(colSums))



In [15]:
gram2_llr = {}
length = len(gram2)
i = 0
for key in gram2:
    if len(key.split()) > 2:
        print(key)
        continue
    gram2_llr[key] = llr(*key.split())
    if i % (int(length / 10)) == 0:
        print(f"{i}/{length}")
    # print(key,gram2_llr[key])
    i += 1

0/2773291
277329/2773291
554658/2773291
831987/2773291
jak:prep na przykład:brev
na przykład:brev kradzież:subst
zgodny:adj to jest:brev
1109316/2773291
urząd:subst do spraw:brev
budynek:subst na przykład:brev
budynek:subst na przykład:brev
nawóz:subst na przykład:brev
na przykład:brev jeżeli:comp
nawóz:subst na przykład:brev
na przykład:brev nie:qub
nawóz:subst na przykład:brev
na przykład:brev w:prep
nawóz:subst na przykład:brev
na przykład:brev zawierawapniowego:adj
częściowo:adv na przykład:brev
na przykład:brev zawierać:pact
nawóz:subst na przykład:brev
na przykład:brev mało:adv
nawóz:subst na przykład:brev
na przykład:brev zawierać:pact
nawóz:subst na przykład:brev
na przykład:brev na:prep
fosforowy:adj to jest:brev
nawóz:subst na przykład:brev
nawóz:subst na przykład:brev
na przykład:brev nie:qub
nawóz:subst na przykład:brev
na przykład:brev jeżeli:comp
nawóz:subst na przykład:brev
na przykład:brev jeżeli:comp
mgo:brev to jest:brev
sód:subst to znaczy:brev
zasadniczy:adj to znac

In [16]:
def sort_dict(dictionary):
    return dict(sorted(dictionary.items(), key=operator.itemgetter(1), reverse=True))


gram2_llr = sort_dict(gram2_llr)
list(gram2_llr.items())[:10]


[('który:adj mowa:subst', 129911.44467048168),
 ('otrzymywać:fin brzmienie:subst', 104889.92928114616),
 ('o:prep który:adj', 98971.94897948248),
 ('w:prep w:prep', 90223.08575718079),
 ('w:prep artykuł:brev', 78823.4428443508),
 ('w:prep ustęp:brev', 63539.375680707126),
 ('mowa:subst w:prep', 50087.93290955135),
 ('dodawać:fin się:qub', 48767.67211949977),
 ('minister:subst właściwy:adj', 46342.51470052161),
 ('i:conj numer:brev', 42964.203399683334)]

8. Partition the entries based on the syntactic categories of the words, i.e. all bigrams having the form of 
   `w1:adj` `w2:subst` should be placed in one partition (the order of the words may not be changed).

In [17]:

gram2 = [f'{i.flexeme} {j.flexeme}' for (i, j) in gram2_tokens]
top10 = Counter(gram2).most_common(10)
top10

[('prep subst', 323768),
 ('subst subst', 280513),
 ('subst adj', 273635),
 ('adj subst', 188202),
 ('subst prep', 171068),
 ('subst conj', 84136),
 ('conj subst', 83096),
 ('ger subst', 81338),
 ('prep adj', 79664),
 ('prep brev', 66986)]

9. Select the 10 largest partitions (partitions with the largest number of entries).
10. Use the computed LLR measure to select 5 bigrams for each of the largest categories.

In [18]:
string = 'otrzymywać:fin brzmienie:subst'
def preprocess(string):
    token1, token2= string.split(" ")
    return (token1.split(":"),token2.split(":"))

def inCategory(string,category):
    ([word1,flexeme1],[word2,flexeme2]) = preprocess(string)
    return category == f"{flexeme1} {flexeme2}"


# preprocess(string)[0][1]
inCategory(string,"fin subst")

True

In [19]:
categories = {i:[] for i, _ in top10}

for bigram,count in gram2_llr.items():
    finished = True
    for category in categories:
        if len(categories[category]) <5:
            finished = False
            if inCategory(bigram,category):
                categories[category].append(bigram+" "+str(count))
    if finished:
        break
            
df=pd.DataFrame.from_dict(categories,orient='index').transpose()

df

,prep subst,subst subst,subst adj,adj subst,subst prep,subst conj,conj subst,ger subst,prep adj,prep brev
0,z:prep dzień:subst 27379.07090054545,droga:subst rozporządzenie:subst 42326.6302884...,minister:subst właściwy:adj 46342.51470052161,który:adj mowa:subst 129911.44467048168,mowa:subst w:prep 50087.93290955135,mowa:subst i:conj 5933.002651984236,i:conj dzień:subst 3805.6834114623653,pozbawić:ger wolność:subst 10748.166422542694,o:prep który:adj 98971.94897948248,w:prep artykuł:brev 78823.4428443508
1,na:prep podstawa:subst 24020.044894876017,skarb:subst państwo:subst 15173.809450670526,rzeczpospolita:subst polski:adj 29972.09246445966,następujący:adj zmiana:subst 14168.634295920887,ustawa:subst z:prep 15977.689100319558,dzień:subst i:conj 3462.6040641931695,i:conj ustawa:subst 3241.9459469478074,zasięgnąć:ger opinia:subst 8210.697539476629,w:prep który:adj 7387.952716919039,w:prep ustęp:brev 63539.375680707126
2,do:prep sprawa:subst 21141.5366789383,rada:subst minister:subst 10169.024629185624,samorząd:subst terytorialny:adj 17812.77131015857,niniejszy:adj ustawa:subst 13170.87462557446,dzień:subst w:prep 8702.908258365422,sprawa:subst i:conj 2868.6403303532406,i:conj sprawa:subst 3038.6657500358597,wykonywać:ger zawód:subst 4038.7485353954557,w:prep właściwy:adj 6000.002702112694,w:prep punkt:brev 7218.804944278644
3,od:prep dzień:subst 18860.003016569382,ochrona:subst środowisko:subst 10093.641427783585,jednostka:subst organizacyjny:adj 17033.457473...,odrębny:adj przepis:subst 7728.815517471113,miesiąc:subst od:prep 7969.428312155529,przepis:subst i:conj 2673.388370419141,lub:conj dzień:subst 2179.500972628529,ograniczyć:ger wolność:subst 3909.855635819227,z:prep który:adj 4426.545214380086,z:prep późniejszy:brev 6559.177105338566
4,w:prep mowa:subst 14610.486845842592,terytorium:subst rzeczpospolita:subst 9974.156...,produkt:subst leczniczy:adj 16029.73146263255,walny:adj zgromadzenie:subst 7243.509665259683,ustawa:subst w:prep 7735.724017269629,ustawa:subst i:conj 2645.9153290615327,i:conj przepis:subst 2160.337295190103,zawrzeć:ger umowa:subst 3661.9406491424647,do:prep który:adj 2693.229605673264,do:prep artykuł:brev 3601.7646793204017


11. Using the results from the previous step answer the following questions:


    i. What types of bigrams have been found? 
    
The most common type of bigram type that was found are substantives (rzeczowniki). Nearly all most numerous categories consist of substantive plus something else 

    
    ii. Which of the category-pairs indicate valuable multiword expressions? Do they have anything in common?
    
I think that the ones which don't have prepositions (prep) or coordinating conjunction (conj) are the ones that we are the most interested in. They often use substantive (rzeczownik) and adjective.
    
    iii. Which signal: LLR score or syntactic category is more useful for determining genuine multiword expressions?
    
LLR together with morphosyntactic category seems to be giving interesting results. It enables us to look at more precise categries eg. subst + adj or ger subst. There are some categories like those with conj that are less interesting for us, and using morphosyntactic category enables us to filter out those categories
    
    iv. Can you describe a different use-case where the morphosyntactic category is useful for resolving a real-world problem?
    
- finding words that tend to appear together eg. "Rzeczpospolita Polska" 
- chatbot - which speaks with a human by changing his words to questions and then asking those questions to maintain a conversation
- translating text